In [1]:
import re
import pandas as pd
import csv
import nltk

In [2]:
def processTweet2(tweet):
    
    tweet=tweet.lower()
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    return tweet

In [3]:
def getStopWordList():
    stop_words=nltk.corpus.stopwords.words('english')
    stop_words.extend(['URL','AT_USER'])
    return set(stop_words)
stop_words=getStopWordList()

In [4]:
def replaceTwoOrMore(s):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)

In [5]:
def getFeatureVector(tweet):
    featureVector = []
    #split tweet into words
    words = nltk.tokenize.word_tokenize(tweet)
    for w in words:
        #replace two or more with two occurrences
        w = replaceTwoOrMore(w)
        #strip punctuation
        w = w.strip('\'"?,.')
        #check if the word stats with an alphabet
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
        #ignore if it is a stop word
        if(w in stop_words or val is None):
            continue
        else:
            featureVector.append(w.lower())
    return featureVector

In [38]:
Gender = pd.read_csv("Path to twitter data", encoding ="ISO-8859-1")
tweets = []
featureList = []
for i in range(len(Gender)):
    sentiment = Gender['gender'][i]
    tweet = Gender['text'][i]
    processedTweet = processTweet2(tweet)
    featureVector = getFeatureVector(processedTweet) 
    featureList.extend(featureVector)
    tweets.append((featureVector, sentiment))

In [39]:
def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in featureList:
        features['contains(%s)' % word] = (word in tweet_words)
    return features
#end
featureList = list(set(featureList))

In [40]:
test=pd.read_csv('Path to twitter data',encoding ="ISO-8859-1")

In [41]:
training_set = nltk.classify.util.apply_features(extract_features, tweets)
# Train the classifier Naive Bayes Classifier
NBClassifier = nltk.NaiveBayesClassifier.train(training_set)
#ua is a dataframe containing all the united airline tweets
test['sentiment'] = test['text'].apply(lambda tweet: 
                                     NBClassifier.classify(extract_features(getFeatureVector(processTweet2(tweet)))))

In [42]:
test.head()

,gender,profile_yn,text,sentiment
0,male,yes,Robbie E Responds To Critics After Win Against...,male
1,male,yes,ÛÏIt felt like they were my friends and I was...,male
2,male,yes,i absolutely adore when louis starts the songs...,male
3,male,yes,Hi @JordanSpieth - Looking at the url - do you...,male
4,female,yes,Watching Neighbours on Sky+ catching up with t...,female


In [43]:
pd.crosstab(test['gender'],test['sentiment'])

sentiment,brand,female,male,unknown
gender,,,,
brand,34,2,3,0
female,1,50,3,0
male,2,1,47,0
unknown,1,1,2,1


In [48]:
(34+50+47+1)/150

0.88

In [45]:
NBClassifier.classify(extract_features(getFeatureVector(processTweet2('This is my life and i will live it on my rules.'))))

'male'

In [46]:
NBClassifier.classify(extract_features(getFeatureVector(processTweet2('The Outing was great good to see you.'))))

'male'

In [47]:
NBClassifier.classify(extract_features(getFeatureVector(processTweet2('Python is great to work with.'))))

'male'